# MLflow Artifactsに保存されるノートブックをインポート
UIで実行したAutoML ExperimentsでもMLflow/Artifact構造は全く同じなので、同じ方法で後から自動取得できます。  
MLflow Runに「generated_notebook.ipynb」が添付されていないケース（トライアルタイプによっては存在しない場合あり）には注意してください。

[AutoML Python API リファレンス](https://learn.microsoft.com/ja-jp/azure/databricks/machine-learning/automl/automl-api-reference#import-notebook)

### カレントディレクトリを取得

In [0]:
current_directory = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get().rsplit('/', 1)[0]
print(current_directory)

### MLflow Artifactsから全Runのノートブックをカレントディレクトリにインポート

In [0]:
# ウィジットを表示
dbutils.widgets.text("experiment_id", "", "Experiment ID")
dbutils.widgets.text("run_id", "", "Run ID")

In [0]:
# ウィジットから入力値を取得
experiment_id = dbutils.widgets.get("experiment_id")
my_run_id = dbutils.widgets.get("run_id")

In [0]:
import mlflow
import databricks.automl

# MLflowから全Runの取得
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(experiment_ids=[experiment_id])

# 各Runのartifact_uriを使ってノートブックをインポート
for run in runs:
    # 取得元のNotebookのアーティファクトパス
    artifact_uri = f"dbfs:/databricks/mlflow-tracking/{experiment_id}/{run.info.run_id}/artifacts/notebooks/training_notebook.ipynb"

    # インポート先のNotebookパス（Notebook名: Run ID）
    notebook_path = f"{current_directory}/automl_notebooks/{run.info.run_id}"

    try:
        result = databricks.automl.import_notebook(
            artifact_uri=artifact_uri,
            path=notebook_path,
            overwrite=True  # 既存の場合は上書き
        )
        print(f"Imported: {result.path} (URL: {result.url})")
    except Exception as e:
        print(f"Failed for run {run.info.run_id}: {e}")

In [0]:
import databricks.automl

'''指定したRunのartifact_uriを使ってノートブックをインポート'''

# 取得元のNotebookのアーティファクトパス
artifact_uri = f"dbfs:/databricks/mlflow-tracking/{experiment_id}/{my_run_id}/artifacts/notebooks/training_notebook.ipynb"

# インポート先のNotebookパス（Notebook名: Run Name + Run ID）
notebook_path = f"{current_directory}/automl_notebooks/{my_run_id}"

try:
    result = databricks.automl.import_notebook(
        artifact_uri=artifact_uri,
        path=notebook_path,
        overwrite=True  # 既存の場合は上書き
    )
    print(f"Imported: {result.path} (URL: {result.url})")
except Exception as e:
    print(f"Failed for run {my_run_id}: {e}")